In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

In [11]:
from src.util import ts_preprocess

In [17]:
from tsfresh import extract_features, select_features, extract_relevant_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction.settings import MinimalFCParameters, ComprehensiveFCParameters
from tsfresh.transformers import RelevantFeatureAugmenter

In [18]:
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

In [2]:
stu_vle = pd.read_csv('data/studentVle.csv')
stu_reg = pd.read_csv('data/studentRegistration.csv')
stu_info = pd.read_csv('data/studentInfo.csv')
ass = pd.read_csv('data/assessments.csv')

In [3]:
module = 'EEE'
pres = '2014J'

In [4]:
stvl_ = stu_vle[stu_vle.code_module.isin([module])][stu_vle.code_presentation.isin([pres])]
stre_ = stu_reg[stu_reg.code_module.isin([module])][stu_reg.code_presentation.isin([pres])]
stin_ = stu_info[stu_info.code_module.isin([module])][stu_info.code_presentation.isin([pres])]

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:
ass_ = ass[ass.code_module.isin([module])][ass.code_presentation.isin([pres])]

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


#### This is a demonstration of how to use ts-fresh for time series feature extraction and/or selection ONLY. (separate from pipeline) To be used in the future with more feature curating.

In [9]:
ass_

,code_module,code_presentation,id_assessment,assessment_type,date,weight
119,EEE,2014J,30719,TMA,33.0,16.0
120,EEE,2014J,30720,TMA,68.0,28.0
121,EEE,2014J,30721,TMA,131.0,28.0
122,EEE,2014J,30722,TMA,166.0,28.0
123,EEE,2014J,30723,Exam,235.0,100.0


In [12]:
X_train, X_test, y_train, y_test, df_train, df_test = ts_preprocess(68, stre_, stin_, stvl_)

In [15]:
df_train.head()

,id_student,date,sum_click
0,60809,0,10
1,60809,3,4
2,60809,4,1
3,60809,7,1
4,60809,8,1


In [19]:
ppl = Pipeline([('fresh', RelevantFeatureAugmenter(column_id='id_student', column_sort='date', 
                                                   default_fc_parameters=ComprehensiveFCParameters())),
                ('clf', RandomForestClassifier(n_estimators = 200,
                                max_depth=5,
                                random_state = 21))])

In [20]:
# for the fit on the train test set, we set the fresh__timeseries_container to `df_train`
ppl.set_params(fresh__timeseries_container=df_train)
ppl.fit(X_train, y_train)

Feature Extraction: 100%|██████████| 20/20 [00:31<00:00,  1.15s/it]


Feature Extraction: 100%|██████████| 20/20 [00:14<00:00,  1.36it/s]


Pipeline(memory=None,
         steps=[('fresh',
                 RelevantFeatureAugmenter(chunksize=None,
                                          column_id='id_student',
                                          column_kind=None, column_sort='date',
                                          column_value=None,
                                          default_fc_parameters={'abs_energy': None,
                                                                 'absolute_sum_of_changes': None,
                                                                 'agg_autocorrelation': [{'f_agg': 'mean',
                                                                                          'maxlag': 40},
                                                                                         {'f_agg': 'median',
                                                                                          'maxlag': 40},
                                                                              

In [21]:
# for the predict on the test test set, we set the fresh__timeseries_container to `df_test`
ppl.set_params(fresh__timeseries_container=df_test)
y_pred = ppl.predict(X_test)

Feature Extraction: 100%|██████████| 20/20 [00:03<00:00,  6.10it/s]


In [22]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.72      0.78        53
           1       0.90      0.95      0.92       142

    accuracy                           0.89       195
   macro avg       0.87      0.83      0.85       195
weighted avg       0.88      0.89      0.88       195

